# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [106]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /home/jc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [87]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('MessageTable', engine)
X = df['message']
Y = df.drop(columns=['id', 'message','original','genre' ])

### 2. Write a tokenization function to process your text data

In [88]:
def tokenize(text):
    tokens = word_tokenize(text)
    
    # Remove Stopwords
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [89]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, np.array(Y), random_state=42)

In [91]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [92]:
y_pred = pipeline.predict(X_test)

In [93]:
y_test.shape

(6554, 36)

In [94]:
y_pred.shape

(6554, 36)

In [95]:
category_names = list(Y.columns.values)

In [96]:
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [97]:
y_test

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [98]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [100]:
for category in range(len(category_names)):
    print('category: {}'.format(category_names[category]))
    print(classification_report(y_test[:, category], y_pred[:, category]))

#print(classification_report(y_test.values, y_pred))
#print(classification_report(y_test, y_pred, target_names=Y.columns.values))

category: related
              precision    recall  f1-score   support

           0       0.62      0.38      0.47      1563
           1       0.82      0.93      0.87      4944
           2       0.54      0.15      0.23        47

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.66      0.48      0.52      6554
weighted avg       0.77      0.79      0.77      6554

category: request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5443
           1       0.81      0.43      0.56      1111

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.85      0.71      0.75      6554
weighted avg       0.88      0.89      0.87      6554

category: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.5

/home/jc/miniconda3/envs/udacity/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6251
           1       0.71      0.14      0.23       303

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.83      0.57      0.60      6554
weighted avg       0.95      0.96      0.94      6554

category: buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6231
           1       0.58      0.08      0.14       323

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.77      0.54      0.56      6554
weighted avg       0.94      0.95      0.93      6554

category: electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6407
           1       0.60      0.02      0.04       147

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.79      0.51      0.51      6554
weighted avg       0.97      0.9

In [105]:
Y['related'].unique()

array([1, 0, 2])

In [102]:
print('Accuracy: {}'.format(np.mean(y_test == y_pred)))

Accuracy: 0.943143288237887


### 6. Improve your model
Use grid search to find better parameters. 

In [117]:
parameters = {
    'clf__estimator__n_estimators': [10,20],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=1, cv=3, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [118]:
tuned_model = cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=10 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: WRITEBACKIFCOPY base is read-only

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.